#### Funciones necesarias para implementar códigos cíclicos torcidos
- right_quo_rem: división a derecha con resto.
- left_quo_rem: división a izquierda con resto.
- left_gcd: máximo común divisor a izquierda.
- right_gcd: máximo común divisor a izquierda.
- left_lcm: mínimo común múltiplo a izquierda.
- right_lcm: mínimo común múltiplo a derecha.
- generalized_right_lcm: mínimo común múltiplo de varios polinomios a derecha.
- generalized_left_lcm: mínimo común múltiplo de varios polinomios a izquierda.

In [75]:
# En esta celda definimos las funciones de division,mcd y mcm

def right_quo_rem(a,b): # a = b*q+r
    q,r = a.left_quo_rem(b)
    return q,r

def left_quo_rem(a,b): #a = q*b+r
    q,r = a.right_quo_rem(b)
    return q,r


def left_gcd(a,b): # f = Xd
    mayor = a
    menor = b
    if b.degree() > a.degree():
        menor = a
        mayor = b
    quo, rem = right_quo_rem(mayor,menor)
    while rem != R(0):
        mayor = menor
        menor = rem
        quo, rem = right_quo_rem(mayor,menor)
    return menor


def right_gcd(a,b): # f = dX d |_r f
    mayor = a
    menor = b
    if b.degree() > a.degree():
        menor = a
        mayor = b
    quo, rem = left_quo_rem(mayor,menor)
    while rem != R(0):
        mayor = menor
        menor = rem
        quo, rem = left_quo_rem(mayor,menor)
    return menor

def right_lcm(a,b):
    f0 = a
    f1 = b
    v0,v1=F(0),F(1)
    qq,rr = right_quo_rem(f0,f1)
    f0=f1
    f1=rr
    v=-qq
    v0=v1
    v1=v
    while f1!=0:
        qq,rr = right_quo_rem(f0,f1)
        f0=f1
        f1=rr
        v=v0 - v1 * qq
        v0=v1
        v1=v
        
    output= b * v1
    #output = output.leading_coefficient()^(-1)*output
    return output


def left_lcm(a,b):
    f0 = a
    f1 = b
    v0,v1=F(0),F(1)
    qq,rr = left_quo_rem(f0,f1)
    f0=f1
    f1=rr
    v=-qq
    v0=v1
    v1=v
    while f1!=0:
        qq,rr = left_quo_rem(f0,f1)
        f0=f1
        f1=rr
        v=v0 - qq * v1
        v0=v1
        v1=v
    output= -v1 * b
    output = output.leading_coefficient()^(-1)*output
    return output

def generalized_right_lcm(ele):
    output = ele[0]
    aux = F(0)
    for ii in range(1,len(ele)):
        aux = right_lcm(output,ele[ii])
        #aux = output.left_lcm(ele[ii])
        output = aux
    return output

def generalized_left_lcm(ele):
    output = ele[0]
    aux = F(0)
    for ii in range(1,len(ele)):
        aux = left_lcm(output,ele[ii])
        output = aux
    return output

### Funciones necesarias para implementar códigos cíclicos torcidos II
- Norma: Norma para evaluar polinomios torcidos.
- REEA: Algoritmo Extendido de Euclides a Derecha.

In [76]:
# Otras funciones necesarias, calculo de la norma y el REEA
def norma(i,a,sigma):
    norma_i = 1
    #Caso base
    if i == 0:
        return norma_i 
    norma_i = norma(i-1,a,sigma)*(sigma^(i-1))(a)
    return norma_i

def REEA(f,g):
    r = []
    r.append(f) ; r.append(g)
    u = []
    u.append(1) ; u.append(0)
    v = []
    v.append(0) ; v.append(1)
    q = 0; rem = 0
    i = 1
    while r[i] != 0:
        q,rem = right_quo_rem(r[i-1],r[i])
        r.append(rem)
        u.append(u[i-1] - u[i]*q)
        v.append(v[i-1] - v[i]*q)
        i = i+1  
    return u,v,r

## Algoritmo de Sugiyama
Pasamos un mensaje codificado que contiene errores y lo decodificamos corrigiendo dichos errores. El algoritmo devuelve el mensaje codificado sin errores.
#### Argumentos
- y: mensaje codificado con errores y = c + e, siendo c una palabra código.
- delta: distancia designada del código, marca la capacidad de correción de errores.
- sigma: automorfimo del cuerpo.
- alpha: elemento que forma una base normal en el cuerpo de invariantes.
#### Pasos:
1. Calculamos los síndromes y el polinomio síndrome S
2. Calculamos un múltiplo del polinomio localizador de errores y del evaluador de errores
3. Calculamos las posiciones de error
4. Calculamos el valor de los errores
5. Obtenemos el polinomio de error


In [77]:
def algoritmo_sugiyama(y,delta,sigma,alpha):
    beta =  sigma(alpha)*alpha^(-1)
    t = floor((delta-1)/2) # número de errores que podemos corregir
    y_j = y.coefficients(sparse=False)

    n = y.degree()
    pol_S = 0
    for i in range(0,2*t): #calculo de los síndromes
        S_i = 0
        auxil = 0
        for j in range(0,n+1):
            aux = 0
            aux = y_j[j]*norma(j,(sigma^i)(beta),sigma)
            S_i = S_i + aux
        auxil = (sigma^i)(alpha)*S_i*x^i 
        pol_S = pol_S + auxil # polinomio sindrome

    
    if pol_S == 0:
        return y
    # REEA para obtener el polinomio localizador, corresponde a la posicion que deg(r_i) < t
    u,v,r = REEA(x^(2*t),pol_S)
    
    
    # Calculamos el j que satsiface deg(r_i) < t
    j = 0

    while r[j].degree() >= t: 
        j = j+1
    
    
    pos_div=[P(x-(sigma^(i-1))(beta^(-1))) for i in range(n+1)]
    pos = []

    for i in range(n+1): # si (sigma^(i-1))(beta^(-1)) es una raiz a izquierda añadimos la posicion i
        q,m = right_quo_rem(v[j],pos_div[i])
        if m == 0 :
            pos.append(i)
    
    if v[j].degree() > len(pos):
        return 'key equation failure'
    p_j = []
    pol_coefs = []
    for i in pos:
        quotient,rem = right_quo_rem(v[j],1-(sigma^i)(beta)*x)
        p_j = quotient
        pol_coefs.append((sigma^i)(alpha)*p_j)
    
    
    # Resolvemos el sistema lineal
    A = matrix(F,r[j].degree()+1)

    for i in range(r[j].degree()+1):
        for k in range(r[j].degree()+1):
            A[k,i] = pol_coefs[i].coefficients(sparse=False)[k]

    VV  = VectorSpace(F,r[j].degree()+1)
    b = VV(r[j].coefficients(sparse=False))

    solution = A.solve_right(b)
    
    # Calculamos el error
    e = 0
    for i in range(len(pos)):
        e = e + solution[i]*x^(pos[i])
    return y-e

### Encuentra posición
Pasamos un múltiplo del polinomio localizador de errores y encontramos una posición de error.

#### Argumentos
- pol: múltiplo del polinomio localizar de errores.
- pos: conjunto tal que grado de p > cardinal(pos).
- sigma: automorfismo del cuerpo.


In [78]:

def encuentra_posicion(pol,pos,sigma):
    pol_f = P(pol)
    grado_e = pol_f.degree()
    for i in range(n+1):
        if i not in pos:
            pol_f = right_lcm(pol_f,P(1-(sigma^i)(beta)*x))
            if pol_f.degree() == grado_e:
                return i
            else:
                grado_e = grado_e +1

### Resolver fallo en la ecuación clave
Pasamos un múltiplo del polinomio localizador de errores y evaluador de errores para obtener justo esos dos polinomios.

#### Argumentos
- v: polinomio múltiplo del localizador de errores.
- r: polinomio múltiplo del evaluador de errores.
- sigma: automorfismo del cuerpo.
- alpha: elemento que forma una base normal en el cuerpo de invariantes.
- n: grado del polinomio recibido y = c + e.

In [79]:
# Calculamos el polinomio localizador de errores y evaluador de errores
def ecuacion_clave_falla(v,r,pos,sigma,alpha,n):
    beta =  sigma(alpha)*alpha^(-1)
    pol_f = P(v)
    while pol_f.degree() > len(pos):
        posicion = encuentra_posicion(pol_f,pos)
        pol_f = right_lcm(pol_f,P(1-(sigma^(posicion))(beta)*x))
        for i in range(n+1):
            if i not in pos:
                cociente,resto = right_quo_rem(pol_f,P(1-(sigma^(i))(beta)*x))
                if resto == 0:
                    pos.append(i)
    pol_g = right_quo_rem(pol_f,v[j])[0]
    return f,r*g


### Generar polinomio generador de un código Reed-Solomon torcido
Obtenemos un polinomio generador con potencias consecutivas de sigma 

#### Argumentos
- alpha: elemento que forma una base normal en el cuerpo de invariantes.
- delta: distancia designada del código.
- sigma: automorfismo del cuerpo.

In [80]:
# Calculamos el polinomio generador de un código Reed-Solomon torcido.
def generar_codigoRS_torcido(alpha,delta,sigma):
    beta = sigma(alpha)*alpha^(-1)

    v = [P(x-(sigma^i)(beta)) for i in range(0,delta-1)] # potencias consecutivas de sigma seguidos
    generador = P(generalized_left_lcm(v))
    return generador

### Ejemplo 

In [81]:
# Definimos el cuerpo que vamos a usar, el automorfismo sigma y la distancia designada delta.
F.<a> = GF(2^7)
sigma = F.frobenius_endomorphism()
n = sigma.order()
P.<x> = F['x',sigma]

delta = 5 # DISTANCIA DESIGNADA




In [82]:
#Calculamos el elemento alpha que forma la base normal eligiendo un elemento aleatorio del cuerpo
# Comprobamos que la matriz tiene determinante distinto de cero ya que es equivalente a ver que 
# forma una base.


alpha = F.random_element()
C = matrix([[(sigma^(i+j))(alpha) for i in range(n)] for j in range(n)])      
while C.determinant() == 0: #necesitamos un elemento cuyo determinante de C no salga cero
    alpha = F.random_element()
    C = matrix([[(sigma^(i+j))(alpha) for i in range(n)] for j in range(n)])
print(alpha)




a^5 + a^3 + 1


In [83]:
# Generamos un polinomio generador del código RS torcido
generador = generar_codigoRS_torcido(alpha,delta,sigma)
print(generador)

x^4 + (a^6 + a^4 + a^3 + a^2 + a + 1)*x^3 + (a^4 + a)*x^2 + (a^6 + a^4 + a^2 + a + 1)*x + a^6 + a^5 + a^3 + a^2 + 1


In [84]:
#Elegimos un mensaje que queremos enviar y lo codificamos
# Elegimos también el error producido y obtenemos la palabra recibida con error que hay que corregir

mensaje = P(x^2+a*x+a^2)
codeword = P(mensaje)*P(generador)
error = P(x^5+a*x^2)


y = P(codeword + error) # Palabra recibida con error por culpa de ruido
print(y)

x^6 + (a^4 + a^3)*x^5 + (a^6 + a^5 + a^2 + a)*x^4 + (a^2 + 1)*x^3 + (a^6 + a^5 + a^4 + a^2 + a)*x^2 + a^3*x + a^5 + a^4 + 1


In [85]:
# Aplicamos el Algoritmo de Sugiyama para corregir los errores
pal_codigo = algoritmo_sugiyama(y,delta,sigma,alpha)

codeword == pal_codigo

True

In [86]:
# Obtenemos el mensaje original decodificando la palabra código corregida
mensaje_original = left_quo_rem(pal_codigo,generador)[0]

mensaje == mensaje_original

True